In [ ]:
#default_exp batch

In [ ]:
#exporti
import pydicom
import matplotlib.pyplot as plt
import os
import sys
import numpy as np
import pandas as pd
import napari
import gif
from fastai.vision import *
import tensorflow.compat.v1 as tf
from misas.core import *
from torchio.transforms import Spike
from misas.mri import *
import altair as alt
import altair_viewer

2022-04-08 10:38:56.021994: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: 
2022-04-08 10:38:56.022071: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Aim: Getting a first impression on the models performance on the given data before going into detailed evaluation

In this case study we demonstrate how `misas` can give you an overview on the performance of the model in form of plots that show the average dice score of the whole batch of images over different parameters:
 - **Model**: [`ukbb_cardiac` network](https://github.com/baiwenjia/ukbb_cardiac) by [Bai et al. 2018 [1]](https://doi.org/10.1186/s12968-018-0471-x), trained on [UK Biobank](https://www.ukbiobank.ac.uk/) cardiac MRI images
 - **Data**: Kaggle [Data Science Bowl Cardiac Challenge Data](https://www.kaggle.com/c/second-annual-data-science-bowl) MRI images

# Prepare Model for Misas

The used model was trained on UK Biobank cardiac imaging data to segment short-axis images of the heart into left ventricle (LV), right ventricle (RV) and myocardium (MY). For details about the model please read [the paper (Bai et al. 2018)](https://doi.org/10.1186/s12968-018-0471-x) and cite it if you use it. For implementation, training and usage see the [GitHub repository](https://github.com/baiwenjia/ukbb_cardiac). We downloaded the pre-trained model for short-axis images from https://www.doc.ic.ac.uk/~wbai/data/ukbb_cardiac/trained_model/ (local copy in `example/kaggle/FCN_sa`). In order to use it with `misas` we need to wrap it in a class that implements the desired interface (`prepareSize` and `predict` taking `Image` as input, see the main docu for more details).

`ukbb_cardiac` is written in `tensorflow` v1. With `tensorflow` v2 make sure to import the compat module.

The model requires images to be a multiple of 16 in each dimension. We pad images accordingly in `prepareSize`. Additionally, code in `image_to_input` takes care of the specifics of transforming a three-channel image into a single-item batch of single-channel images. In `predict` the output is converted to `ImageSegment` class.

In [ ]:
class ukbb_model:
    def __init__(self, model_path):
        tf.disable_eager_execution()
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        saver = tf.train.import_meta_graph(f'{model_path}.meta')
        saver.restore(self.sess, model_path)
        
    def prepareSize(self, image):
        _, X, Y = image.shape
        image.crop_pad((int(math.ceil(X / 16.0)) * 16, int(math.ceil(Y / 16.0)) * 16), padding_mode="zeros")
        return image
    
    def image_to_input(self, image):
        img = image.clone()
        self.prepareSize(img)
        img_data = img.data[0]
        img_data = np.expand_dims(img_data, 0)
        img_data = np.expand_dims(img_data, -1)
        return img_data
    
    def predict(self, image):
        image_data = self.image_to_input(image)
        preds, classes = self.sess.run(['prob:0', 'pred:0'],
                   feed_dict={'image:0': image_data, 'training:0': False})
        preds = np.squeeze(preds, 0)
        classes = ImageSegment(ByteTensor(classes))
        return classes, preds

In [ ]:
model = ukbb_model('example/kaggle/FCN_sa')

2022-04-08 10:39:22.025924: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: 
2022-04-08 10:39:22.025985: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-08 10:39:22.026048: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (gaia4): /proc/driver/nvidia/version does not exist
2022-04-08 10:39:22.730860: W tensorflow/core/common_runtime/graph_constructor.cc:1511] Importing a graph with a lower producer version 24 into an existing graph with producer version 987. Shape inference will have run different parts of the graph with different producer versions.


INFO:tensorflow:Restoring parameters from example/kaggle/FCN_sa


# Prepare Images

In [ ]:
#exporti
def prepareImage(file):
    ds = pydicom.dcmread(file)
    img = Tensor(ds.pixel_array.astype(np.int16))
    img = img/img.max()
    img = Image(torch.stack([img, img, img]))    # statt: img = Image(img.unsqueeze(0))
    img = img.flip_lr()
    img= img.rotate(90)
    return img

In [ ]:
files = ["example/kaggle/sample_images/IM-13717-0026.dcm",
        "example/kaggle/sample_images/IM-7453-0024.dcm",
        "example/kaggle/sample_images/IM-4718-0021.dcm",
        "example/kaggle/sample_images/IM-5022-0015.dcm",
        "example/kaggle/sample_images/IM-14141-0011.dcm",
        "example/kaggle/sample_images/IM-13811-0003.dcm",
        "example/kaggle/sample_images/IM-5382-0008.dcm"]

# Evaluation

The `batch_results` function takes 4 arguments:

- `images`: a list of paths for dicom files with which the model should be evaluated
- `model`: the model that should be evaluated
- `transformation_functions`: the names of the functions that should be evaluated (eval functions from misas.core)
- `true_masks`:
    - a list of png files with the true masks for "images" ***in the same order as "images"***
    - if true_masks is "None", the model will predict a mask for every image, save it as png in a newly created directory and use this mask as truth for the evaluation and afterwards delete the directory and the contained predicted thruths. Thus, it is important to hand in the images in an orientation in which the model makes good predictions, in this case this is handled, by the "prepareImage" function, which reads the dicom file, converts it to a fastai Image object, and rotates and flips the image into correct position

`batch_results` returns a list of Pandas dataFrames, that contains one dataFrame for each image with the columns: parameter, bg, LV, MY, RV, File.

In [ ]:
#export
def batch_results(images, model, transformation_functions, true_masks=None):
    results = []
    for x in transformation_functions:
        trfm_result = []
        for index, i in enumerate(images):
            img = lambda: prepareImage(i)
            if true_masks == None:
                path = os.path.join(os.getcwd(), "predicted_truths")
                os.mkdir(path)
                truth = model.predict(img())[0]
                truth.save(os.path.join(path, "current_truth.png"))
                true_mask = lambda: open_mask(os.path.join(path, "current_truth.png"))
            elif true_masks != None:
                true_mask = lambda: open_mask(true_mask[index])
            df = x(img(), true_mask(), model, components=['bg','LV','MY', "RV"])
            df["File"] = i
            trfm_result.append(df)
            shutil.rmtree(path)
        trfm_result = pd.concat(trfm_result)
        results.append(trfm_result)
    return results

In [ ]:
results = batch_results(files, model, [eval_rotation_series,
                                       eval_bright_series,
                                       eval_crop_series,
                                       eval_contrast_series,
                                       eval_dihedral_series,
                                       eval_resize_series,
                                       eval_spike_series,
                                       eval_spike_pos_series,
                                       eval_zoom_series])
results

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

...

  0%|          | 0/45 [00:00<?, ?it/s]

  0%|          | 0/45 [00:00<?, ?it/s]

  0%|          | 0/45 [00:00<?, ?it/s]

  0%|          | 0/45 [00:00<?, ?it/s]

[    deg        bg        LV        MY        RV  \
 0     0  1.000000  1.000000  1.000000  1.000000   
 1     5  0.998885  0.986046  0.944737  0.936047   
 2    10  0.998906  0.988399  0.951743  0.935461   
 3    15  0.998821  0.986007  0.950535  0.939306   
 4    20  0.998532  0.982751  0.935099  0.916914   
 ..  ...       ...       ...       ...       ...   
 67  335  0.999307  0.974006  0.941824  0.979928   
 68  340  0.999264  0.976318  0.942109  0.973806   
 69  345  0.999382  0.981017  0.954941  0.977296   
 70  350  0.999541  0.978659  0.951779  0.985729   
 71  355  0.999318  0.977238  0.945860  0.977413   
 
                                               File  
 0   example/kaggle/sample_images/IM-13717-0026.dcm  
 1   example/kaggle/sample_images/IM-13717-0026.dcm  
 2   example/kaggle/sample_images/IM-13717-0026.dcm  
 3   example/kaggle/sample_images/IM-13717-0026.dcm  
 4   example/kaggle/sample_images/IM-13717-0026.dcm  
 ..                                             ..

# Plotting the results
The `plot_batch` function displays the plots and also returns a list that contains the plots as altair.FacetChart objects. It takes two arguments:
- a list of dataframes which contains one dataFrame for each transformation in a format as it is returned by `batch_results` (columns: parameter, bg, LV, MY, RV, File)
- plottype:
    - avg_errorbars: plots the average of the dice score and shows the standarddeviation as errorbars
    - avg_dots: plots the average of the dice score and additionally shows the single datapoints instead of errorbars
    - boxplot

In [ ]:
#export
def plot_batch(df_results, plottype="avg_errorbars"):
    plots = []
    for i in df_results:
        if plottype == "avg_errorbars":
            plot = plot_avg_and_errorbars(i)
        elif plottype == "avg_dots":
            plot = plot_avg_and_dots(i)
        elif plottype == "boxplot":
            plot = plot_boxplot(i)
        elif plottype == "area":
            plot = plot_area(i)
        else:
            raise ValueError(f'plottype must be one of "avg_errorbars", "avg_dots", "area" or "boxplot"')
        plots.append(plot)
    for p in plots:
        p.display()
    return plots

In [ ]:
#export
def plot_avg_and_dots(df):
    melt_results = df.melt(id_vars=df.columns[0], value_vars=df.columns[2:5])
    avg_line_plot = alt.Chart(melt_results
                ).mark_line(
                ).encode(x=melt_results.columns[0], y="average(value)", color=alt.Color("variable")
                ).properties(width=400, height=200
                ).interactive()
    dot_plot = alt.Chart(melt_results
                ).mark_point(
                ).encode(x=melt_results.columns[0], y="value", color=alt.Color("variable")
                ).properties(width=400, height=200
                ).interactive()
    plot = alt.layer(dot_plot, avg_line_plot).facet(column="variable")
    return plot

In [ ]:
#export
def plot_avg_and_errorbars(df):
    melt_results = df.melt(id_vars=df.columns[0], value_vars=df.columns[2:5])
    avg_line_plot = alt.Chart(melt_results
                ).mark_line(
                ).encode(x=melt_results.columns[0], y="average(value)", color=alt.Color("variable")
                ).properties(width=400, height=200
                ).interactive()
    error_bars = alt.Chart(melt_results
                ).mark_errorbar(extent='stdev'
                ).encode(x=melt_results.columns[0], y="value", color=alt.Color("variable"))
    plot = alt.layer(avg_line_plot, error_bars).facet(column="variable")
    return plot

In [ ]:
#export
def plot_area(df):
    melt_results = df.melt(id_vars=df.columns[0], value_vars=df.columns[2:5])
    area = alt.Chart(melt_results
                    ).mark_area(opacity=0.3, color='#57A44C'
                    ).encode(x=melt_results.columns[0], y="value", color=alt.Color("variable")
                    ).properties(width=400, height=200
                    ).interactive()
    avg_line_plot = alt.Chart(melt_results
                ).mark_line(
                ).encode(x=melt_results.columns[0], y="average(value)", color=alt.Color("variable")
                ).properties(width=400, height=200
                ).interactive()
    plot = alt.layer(area, avg_line_plot).facet(column="variable")
    return plot

In [ ]:
#export
def plot_boxplot(df):
    melt_results = df.melt(id_vars=df.columns[0], value_vars=df.columns[2:5])
    plot = alt.Chart(melt_results
                ).mark_boxplot(extent="min-max", size=5
                ).encode(x=alt.X(melt_results.columns[0]), y=alt.Y("value"), color=alt.Color("variable")
                ).properties(width=400, height=200
                ).facet(column="variable"
                ).interactive()
    return plot

In [ ]:
plots = plot_batch(results, plottype="avg_dots")

alt.FacetChart(...)

alt.FacetChart(...)

alt.FacetChart(...)

alt.FacetChart(...)

alt.FacetChart(...)

alt.FacetChart(...)

alt.FacetChart(...)

alt.FacetChart(...)

alt.FacetChart(...)